In [1]:
import dask.dataframe as dd
from dask.dataframe.utils import make_meta
from neo4j import GraphDatabase
from dask.distributed import Client, LocalCluster, get_worker
import dask
import dill
import os

In [2]:
def addActivity(tx, activityName):
    return tx.run("MERGE (p:Activity {name: $activityName})", activityName=activityName)

def setLinks(row):
    row['predecessor'] = row['activityNameEN'].shift(1); #lag(1)
    row['successor'] = row['activityNameEN'].shift(-1); #lead(1)
    return row;

def getNeo4jCredentials():
    return {
        "host": "bolt://44.200.240.194:7687",
        "user": "neo4j",
        "password": "liter-choke-sizing"
    }

In [3]:
columnTypes = {
    'case:IDofConceptCase': 'string',
    'case:Includes_subCases': 'string',
    'case:Responsible_actor': 'string',
    'case:caseProcedure': 'string',
    'dateStop': 'string'
}
df = dd.read_csv('BPIC15_1.csv', dtype=columnTypes)

In [4]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 5,Total memory: 7.67 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33483,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 5
Started: Just now,Total memory: 7.67 GiB
Comm: tcp://127.0.0.1:36873,Total threads: 1
Dashboard: http://127.0.0.1:35613/status,Memory: 1.53 GiB
Nanny: tcp://127.0.0.1:40649,


In [5]:
creds = getNeo4jCredentials()

In [6]:
def experiment(result):
    driver = GraphDatabase.driver(creds.get('host'), auth=(creds.get('user'), creds.get('password')))
#     with driver.session() as session:
#         result.apply(lambda activityName: session.run("MERGE (p:Activity {name: $activityName})", activityName=activityName),
#                      meta=('activityNameEN', 'object')).compute()
#     session.close()
#     driver.close()
    return result

In [7]:
df['successor'] = ''
df['predecessor'] = ''
df = df.sort_values(by='time:timestamp').groupby('case:concept:name').apply(setLinks, meta=df)

In [8]:
result = df['activityNameEN'].unique()

In [9]:
f_result = client.scatter(result, broadcast=True)

In [10]:
output = client.submit(experiment, f_result)

In [11]:
x = output.result()

In [12]:
def test(activityName):
    driver = GraphDatabase.driver(creds.get('host'), auth=(creds.get('user'), creds.get('password')))
    with driver.session() as session:
        session.run("MERGE (p:Activity {name: $activityName})", activityName=activityName)
        session.close()
    return activityName

In [13]:
driver = GraphDatabase.driver(creds.get('host'), auth=(creds.get('user'), creds.get('password')))
x.apply(test, meta=('activityNameEN', 'object')).compute()
# with driver.session() as session:
# #     session.run("MERGE (p:Activity {name: $activityName})", activityName="Test_Activity2")
#     x[0].apply(test, meta=('activityNameEN', 'object'), session=dill.dumps(session)).compute()
# #     session.close()
# driver.close()

2022-12-09 17:01:16,270 - bokeh.application.handlers.function - ERROR - '<' not supported between instances of 'NoneType' and 'tuple'
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/distributed/utils.py", line 784, in wrapper
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.9/site-packages/distributed/dashboard/components/scheduler.py", line 3894, in graph_doc
    graph = TaskGraph(scheduler, sizing_mode="stretch_both")
  File "/opt/conda/lib/python3.9/site-packages/distributed/dashboard/components/scheduler.py", line 2076, in __init__
    self.layout = GraphLayout(scheduler)
  File "/opt/conda/lib/python3.9/site-packages/distributed/diagnostics/graph_layout.py", line 41, in __init__
    self.update_graph(
  File "/opt/conda/lib/python3.9/site-packages/distributed/diagnostics/graph_layout.py", line 51, in update_graph
    stack = sorted(tasks, key=lambda k: priority.get(k, 0), reverse=True)
TypeError: '<' not supported between instances 

0                   register submission date request
1                         phase application received
2              enter senddate procedure confirmation
3      registrer date of publishing received request
4                     enter senddate acknowledgement
                           ...                      
284                        phase draft decision sent
285                         appealed to higher court
286            create letter requesting updated plan
287                     phase updated plan requested
288                      phase updated plan received
Name: activityNameEN, Length: 289, dtype: object

In [14]:
x[0].compute()

0    register submission date request
Name: activityNameEN, dtype: object